<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula07c_mnist_flat_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [13]:
train_flat_images = train_images.reshape((60000, 28 * 28))
test_flat_images = test_images.reshape((10000, 28 * 28))
print(train_flat_images.shape)

(60000, 784)


In [14]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [15]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

train_flat_images_tensor = torch.tensor(train_flat_images, dtype=torch.float32).to(device)
train_labels_torch = torch.tensor(train_labels).to(device)
test_flat_images_tensor = torch.tensor(test_flat_images, dtype=torch.float32).to(device)
test_labels_torch = torch.tensor(test_labels).to(device)

train_dataset = TensorDataset(train_flat_images_tensor, train_labels_torch)
test_dataset = TensorDataset(test_flat_images_tensor, test_labels_torch)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

class BasicTorchNN(nn.Module):
  def __init__(self, num_classes):
    super(BasicTorchNN, self).__init__()
    self.fc1 = nn.Linear(28 * 28, 512)
    self.fc2 = nn.Linear(512, num_classes)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = BasicTorchNN(10).to(device)
optimizer = optim.RMSprop(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 5
batch_size = 128

for epoch in range(epochs):
  for batch in train_loader:
    images, labels = batch
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

with torch.no_grad():
  outputs = model(images)
  _, predicted = torch.max(outputs.data, 1)
  total = labels.size(0)
  correct = (predicted == labels).sum().item()
  print(f"Accuracy: {correct / total}")


Epoch 1/5, Loss: 0.31326568126678467
Epoch 2/5, Loss: 0.10025376826524734
Epoch 3/5, Loss: 0.15682794153690338
Epoch 4/5, Loss: 0.26227349042892456
Epoch 5/5, Loss: 0.042041029781103134
Accuracy: 0.9791666666666666


In [16]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
import numpy as np

class TorchWrappedNN(BaseEstimator, ClassifierMixin):
  def __init__(self, epochs=5, batch_size=128, model_fabric=BasicTorchNN):
    self.epochs = epochs
    self.batch_size = batch_size
    self.model_fabric = model_fabric

  def fit(self, X, y):
    self.labels, ids = np.unique(y, return_inverse=True)

    ytensor = torch.tensor(ids, dtype=torch.long).to(device)
    xtensor = torch.tensor(X, dtype=torch.float32).to(device)
    dataset = TensorDataset(xtensor, ytensor)
    loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    self.model = self.model_fabric(len(self.labels)).to(device)
    self.optimizer = optim.RMSprop(self.model.parameters(), lr=0.001)
    self.criterion = nn.CrossEntropyLoss()

    for epoch in range(self.epochs):
      for batch in loader:
        images, labels = batch
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
      print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {loss.item()}")
    return self

  def predict(self, X):
    with torch.no_grad():
      xtensor = torch.tensor(X, dtype=torch.float32).to(device)
      outputs = self.model(xtensor)
      _, predicted = torch.max(outputs.data, 1)
      return self.labels[predicted.cpu().numpy()]

model = TorchWrappedNN(epochs=5, batch_size=128)
model.fit(train_flat_images, train_labels)
y_pred = model.predict(test_flat_images)
accuracy_score(test_labels, y_pred)

Epoch 1/5, Loss: 0.5393354296684265
Epoch 2/5, Loss: 0.26431915163993835
Epoch 3/5, Loss: 0.0323740616440773
Epoch 4/5, Loss: 0.11840377002954483
Epoch 5/5, Loss: 0.06080253794789314


0.9666

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", TorchWrappedNN())
])

pipeline.fit(train_flat_images, train_labels)
y_pred = pipeline.predict(test_flat_images)
accuracy_score(test_labels, y_pred)

Epoch 1/5, Loss: 0.1512201875448227
Epoch 2/5, Loss: 0.10874402523040771
Epoch 3/5, Loss: 0.015678133815526962
Epoch 4/5, Loss: 0.02246454916894436
Epoch 5/5, Loss: 0.007748025003820658


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


0.9734

In [18]:
from sklearn.base import TransformerMixin

class Divide255(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X / 255.0

pipeline = Pipeline([
    ("scaler", Divide255()),
    ("model", TorchWrappedNN())
])

pipeline.fit(train_flat_images, train_labels)
y_pred = pipeline.predict(test_flat_images)
accuracy_score(test_labels, y_pred)

Epoch 1/5, Loss: 0.09070269018411636
Epoch 2/5, Loss: 0.06805183738470078
Epoch 3/5, Loss: 0.033073168247938156
Epoch 4/5, Loss: 0.06653110682964325
Epoch 5/5, Loss: 0.04836578294634819


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


0.9802

In [19]:
class Shape2Flat(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X.reshape((-1, 28 * 28))

pipeline = Pipeline([
    ("shape2flat", Shape2Flat()),
    ("scaler", Divide255()),
    ("model", TorchWrappedNN())
])

pipeline.fit(train_images, train_labels)
y_pred = pipeline.predict(test_images)
accuracy_score(test_labels, y_pred)

Epoch 1/5, Loss: 0.08692585676908493
Epoch 2/5, Loss: 0.09133532643318176
Epoch 3/5, Loss: 0.07623925805091858
Epoch 4/5, Loss: 0.0263067539781332
Epoch 5/5, Loss: 0.09983951598405838


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


0.9749